In [1]:
import pandas as pd
from pathlib import Path

In [2]:
DATA_DIR = Path("data/processing_checkpoint")
path_clean_dataset = DATA_DIR / "01_cleaned_train.parquet"

In [3]:
df_clean_churn = pd.read_parquet(path_clean_dataset)

In [4]:
df_clean_churn.sample(5)

,gender,level,userId,page,sessionId,itemInSession,length,song,artist,time,registration,metropolitan_area,region,operating_system,browser
4889437,M,paid,1144779,NextSong,124556,164,211.06893,Roll it Gal,Alison Hinds,2018-10-29 06:26:58,2018-08-13 08:33:29,Lexington-Fayette,KY,Macintosh,Safari
9906831,M,paid,1214440,NextSong,40348,11,201.79546,Revelry,Kings Of Leon,2018-10-06 00:49:16,2018-09-24 04:00:51,Tulsa,OK,Macintosh,Safari
3088651,M,paid,1470782,NextSong,104633,126,285.88363,Shine,Blue Foundation,2018-10-22 21:33:52,2018-05-08 23:26:51,Charlotte-Concord-Gastonia,NC-SC,Macintosh,Chrome
5493527,F,paid,1880582,NextSong,184105,116,235.83302,Bouncing Around The Room (LP Version),Phish,2018-11-13 02:09:49,2018-08-15 12:23:56,Boulder,CO,Linux,Chrome
5320837,F,paid,1991682,NextSong,94437,53,137.87383,Remember (Walking In The Sand),The Shangri-Las,2018-10-19 20:57:02,2018-08-26 20:30:58,San Jose-Sunnyvale-Santa Clara,CA,Windows,Edge


In [5]:
df_users_base = df_clean_churn[["userId", "gender", "registration", "operating_system", "browser"]].drop_duplicates(subset=["userId"])

In [6]:
len(df_users_base)

19140

In [7]:
df_users_base = df_users_base.set_index("userId")

In [13]:
df_unique_artists = (
    df_clean_churn.groupby("userId")["artist"].nunique().reset_index().rename(columns={"artist": "num_unique_artists"})
)
df_unique_artists = df_unique_artists.set_index("userId")
df_users_base = df_users_base.join(df_unique_artists)

In [15]:
df_users_base.sample(5)

,registration,operating_system,browser,num_unique_artists
userId,,,,
1495991,2018-08-17 02:10:47,Windows,Firefox,1320
1484921,2018-09-16 09:11:42,Linux,Chrome,465
1813752,2018-08-31 09:06:46,Windows,Chrome,725
1231449,2018-09-16 09:28:41,Macintosh,Chrome,625
1923638,2018-09-21 12:43:10,Windows,Chrome,582
